<a href="https://colab.research.google.com/github/chardiwall/DPSNN/blob/main/DPSNN_Brain_Cog.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q tonic
!pip install -q opacus
!pip install -q braincog
!pip install -q timm

ERROR: Operation cancelled by user


# Libraries

In [ ]:
import numpy as np
from torchvision import datasets, transforms
import torch
from torch.utils.data import Dataset
import tonic
from tonic import DiskCachedDataset
import torch.nn.functional as F
import os

import abc
from functools import partial
import torch
from torch.nn import functional as F
import torchvision
from timm.models import register_model
from braincog.base.node.node import *
from braincog.base.connection.layer import *
from braincog.base.encoder.encoder import *
from braincog.model_zoo.base_module import BaseModule, BaseConvModule, BaseLinearModule

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from opacus import PrivacyEngine


from braincog.base.node.node import *
import warnings
from opacus.utils.batch_memory_manager import BatchMemoryManager

warnings.simplefilter("ignore")

/usr/local/lib/python3.10/dist-packages/braincog/model_zoo/resnet.py:461: UserWarning: Overwriting resnet18 in registry with braincog.model_zoo.resnet.resnet18. This is because the name being registered conflicts with an existing name. Please check if this is not expected.
  def resnet18(pretrained=False, **kwargs):
/usr/local/lib/python3.10/dist-packages/braincog/model_zoo/resnet.py:474: UserWarning: Overwriting resnet34 in registry with braincog.model_zoo.resnet.resnet34. This is because the name being registered conflicts with an existing name. Please check if this is not expected.
  def resnet34(pretrained=False, **kwargs):
/usr/local/lib/python3.10/dist-packages/braincog/model_zoo/resnet.py:486: UserWarning: Overwriting resnet50 in registry with braincog.model_zoo.resnet.resnet50. This is because the name being registered conflicts with an existing name. Please check if this is not expected.
  def resnet50(pretrained=False, **kwargs):
/usr/local/lib/python3.10/dist-packages/brainc

# Data

In [ ]:
MNIST_MEAN = 0.1307
MNIST_STD = 0.3081
CIFAR10_MEAN = (0.4914, 0.4822, 0.4465)
CIFAR10_STD_DEV = (0.2023, 0.1994, 0.2010)
cifar100_mean = [0.5071, 0.4865, 0.4409]
cifar100_std = [0.2673, 0.2563, 0.2761]
DVSCIFAR10_MEAN_16 = [0.3290, 0.4507]
DVSCIFAR10_STD_16 = [1.8398, 1.6549]

DATA_DIR = '/data/datasets'


class CustomDataset(Dataset):
    """An abstract Dataset class wrapped around Pytorch Dataset class.
    """

    def __init__(self, dataset, indices):
        self.dataset = dataset
        self.indices = [int(i) for i in indices]

    def __len__(self):
        return len(self.indices)

    def __getitem__(self, item):
        x, y = self.dataset[self.indices[item]]
        return x, y


def load_static_data(data_root, batch_size, dataset):
    if dataset == 'cifar10':
        transform_train = transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize(CIFAR10_MEAN, CIFAR10_STD_DEV)])

        transform_test = transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize(CIFAR10_MEAN, CIFAR10_STD_DEV)])

        train_data = datasets.CIFAR10(data_root, train=True, transform=transform_train, download=True)
        test_data = datasets.CIFAR10(data_root, train=False, transform=transform_test, download=True)

        train_loader = torch.utils.data.DataLoader(
            train_data,
            batch_size=batch_size,
            shuffle=True
        )
        test_loader = torch.utils.data.DataLoader(
            test_data,
            batch_size=batch_size,
        )
    elif dataset == 'MNIST':
        transform_train = transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize(MNIST_MEAN, MNIST_STD)])

        transform_test = transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize(MNIST_MEAN, MNIST_STD)])

        train_data = datasets.MNIST(data_root, train=True, transform=transform_train, download=True)
        test_data = datasets.MNIST(data_root, train=False, transform=transform_test, download=True)

        train_loader = torch.utils.data.DataLoader(
            train_data,
            batch_size=batch_size,
            shuffle=True
        )
        test_loader = torch.utils.data.DataLoader(
            test_data,
            batch_size=batch_size,
        )
    elif dataset == 'FashionMNIST':
        transform_train = transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize(MNIST_MEAN, MNIST_STD)])

        transform_test = transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize(MNIST_MEAN, MNIST_STD)])

        train_data = datasets.FashionMNIST(data_root, train=True, transform=transform_train, download=True)
        test_data = datasets.FashionMNIST(data_root, train=False, transform=transform_test, download=True)

        train_loader = torch.utils.data.DataLoader(
            train_data,
            batch_size=batch_size,
            shuffle=True
        )
        test_loader = torch.utils.data.DataLoader(
            test_data,
            batch_size=batch_size,
        )

    return train_data, test_data, train_loader, test_loader


def load_dvs10_data(batch_size, step, **kwargs):
    size = kwargs['size'] if 'size' in kwargs else 48
    sensor_size = tonic.datasets.CIFAR10DVS.sensor_size
    train_transform = transforms.Compose([
        # tonic.transforms.Denoise(filter_time=10000),
        # tonic.transforms.DropEvent(p=0.1),
        tonic.transforms.ToFrame(sensor_size=sensor_size, n_time_bins=step), ])
    test_transform = transforms.Compose([
        # tonic.transforms.Denoise(filter_time=10000),
        tonic.transforms.ToFrame(sensor_size=sensor_size, n_time_bins=step), ])
    train_dataset = tonic.datasets.CIFAR10DVS(os.path.join(DATA_DIR, 'DVS/DVS_Cifar10'), transform=train_transform)
    test_dataset = tonic.datasets.CIFAR10DVS(os.path.join(DATA_DIR, 'DVS/DVS_Cifar10'), transform=test_transform)

    train_transform = transforms.Compose([
        lambda x: torch.tensor(x, dtype=torch.float),
        lambda x: F.interpolate(x, size=[size, size], mode='bilinear', align_corners=True),
    ])
    test_transform = transforms.Compose([
        lambda x: torch.tensor(x, dtype=torch.float),
        lambda x: F.interpolate(x, size=[size, size], mode='bilinear', align_corners=True),
    ])

    train_dataset = DiskCachedDataset(train_dataset,
                                      cache_path=f'./dataset/dvs_cifar10/train_cache_{step}',
                                      transform=train_transform)
    test_dataset = DiskCachedDataset(train_dataset,
                                     cache_path=f'./dataset/dvs_cifar10/test_cache_{step}',
                                     transform=test_transform)

    num_train = len(train_dataset)
    num_per_cls = num_train // 10
    indices_train, indices_test = [], []
    portion = kwargs['portion'] if 'portion' in kwargs else .9
    for i in range(10):
        indices_train.extend(
            list(range(i * num_per_cls, round(i * num_per_cls + num_per_cls * portion))))
        indices_test.extend(
            list(range(round(i * num_per_cls + num_per_cls * portion), (i + 1) * num_per_cls)))
    train_dataset = CustomDataset(train_dataset, np.array(indices_train))
    test_dataset = CustomDataset(test_dataset, np.array(indices_test))

    train_loader = torch.utils.data.DataLoader(
        train_dataset, batch_size=batch_size, shuffle=True,
        pin_memory=True, drop_last=False, num_workers=1
    )

    test_loader = torch.utils.data.DataLoader(
        test_dataset, batch_size=batch_size,
        pin_memory=True, drop_last=False, num_workers=1
    )

    return train_loader, test_loader, train_dataset, test_dataset


def load_nmnist_data(batch_size, step, **kwargs):
    size = kwargs['size'] if 'size' in kwargs else 28
    sensor_size = tonic.datasets.NMNIST.sensor_size
    train_transform = transforms.Compose([
        # tonic.transforms.Denoise(filter_time=10000),
        # tonic.transforms.DropEvent(p=0.1),
        tonic.transforms.ToFrame(sensor_size=sensor_size, n_time_bins=step), ])
    test_transform = transforms.Compose([
        # tonic.transforms.Denoise(filter_time=10000),
        tonic.transforms.ToFrame(sensor_size=sensor_size, n_time_bins=step), ])
    train_dataset = tonic.datasets.NMNIST(os.path.join(DATA_DIR, 'DVS/NMNIST'), transform=train_transform, train=True)
    test_dataset = tonic.datasets.NMNIST(os.path.join(DATA_DIR, 'DVS/NMNIST'), transform=test_transform, train=False)

    train_transform = transforms.Compose([
        lambda x: torch.tensor(x, dtype=torch.float),
        lambda x: F.interpolate(x, size=[size, size], mode='bilinear', align_corners=True),

    ])
    test_transform = transforms.Compose([
        lambda x: torch.tensor(x, dtype=torch.float),
        lambda x: F.interpolate(x, size=[size, size], mode='bilinear', align_corners=True),
    ])

    train_dataset = DiskCachedDataset(train_dataset,
                                      cache_path=f'./dataset/NMNIST/train_cache_{step}',
                                      transform=train_transform)
    test_dataset = DiskCachedDataset(test_dataset,
                                     cache_path=f'./dataset/NMNIST/test_cache_{step}',
                                     transform=test_transform)

    train_loader = torch.utils.data.DataLoader(
        train_dataset, batch_size=batch_size, shuffle=True,
        pin_memory=True, drop_last=False, num_workers=1
    )

    test_loader = torch.utils.data.DataLoader(
        test_dataset, batch_size=batch_size,
        pin_memory=True, drop_last=False, num_workers=1
    )

    return train_loader, test_loader, train_dataset, test_dataset

# Model

In [ ]:
class TEP(nn.Module):
    def __init__(self, step, channel, device=None, dtype=None):
        factory_kwargs = {'device': device, 'dtype': dtype}
        super(TEP, self).__init__()
        self.step = step
        self.gn = nn.GroupNorm(channel, channel)


    def forward(self, x):

        x = rearrange(x, '(t b) c w h -> t b c w h', t=self.step)
        fire_rate = torch.mean(x, dim=0)
        fire_rate = self.gn(fire_rate) + 1

        x = x * fire_rate
        x = rearrange(x, 't b c w h -> (t b) c w h')

        return x


class BaseConvNet(BaseModule, abc.ABC):
    def __init__(self,
                 step,
                 input_channels,
                 num_classes,
                 encode_type,
                 spike_output: bool,
                 out_channels: list,
                 block_depth: list,
                 node_list: list,
                 *args,
                 **kwargs):
        super().__init__(step, encode_type, *args, **kwargs)
        self.num_cls = num_classes
        self.spike_output = spike_output
        self.groups = kwargs['n_groups'] if 'n_groups' in kwargs else 1
        if not spike_output:
            node_list.append(nn.Identity)
            out_channels.append(self.num_cls)
            self.vote = nn.Identity()
            # self.vote = nn.Sequential(
            #     nn.Linear(self.step, 32),
            #     nn.ReLU(),
            #     nn.Linear(32, 1)
            # )
        else:
            out_channels.append(10 * self.num_cls)
            self.vote = VotingLayer(10)

        # check list length
        if len(node_list) != len(out_channels):
            raise ValueError
        self.input_channels = input_channels
        self.out_channels = out_channels
        self.block_depth = block_depth
        self.node_list = node_list
        self.feature = self._create_feature()
        self.fc = self._create_fc()
        if self.layer_by_layer:
            self.flatten = nn.Flatten(start_dim=1)
        else:
            self.flatten = nn.Flatten()

    @staticmethod
    def _create_feature(self):
        raise NotImplementedError

    @staticmethod
    def _create_fc(self):
        raise NotImplementedError

    def forward(self, inputs):
        inputs = self.encoder(inputs)
        self.reset()
        if not self.training:
            self.fire_rate.clear()

        if not self.layer_by_layer:
            outputs = []
            if self.warm_up:
                step = 1
            else:
                step = self.step

            for t in range(step):
                x = inputs[t]
                x = self.feature(x)
                x = self.flatten(x)
                x = self.fc(x)
                x = self.vote(x)
                outputs.append(x)

            return sum(outputs) / len(outputs)
            # outputs = torch.stack(outputs)
            # outputs = rearrange(outputs, 't b c -> b c t')
            # outputs = self.vote(outputs).squeeze()
            # return outputs

        else:
            x = self.feature(inputs)
            x = self.flatten(x)
            x = self.fc(x)
            if self.groups == 1:
                x = rearrange(x, '(t b) c -> t b c', t=self.step).mean(0)
            else:
                x = rearrange(x, 'b (c t) -> t b c', t=self.step).mean(0)
            x = self.vote(x)
            return x


class LayerWiseConvModule(nn.Module):
    """
    SNN卷积模块
    :param in_channels: 输入通道数
    :param out_channels: 输出通道数
    :param kernel_size: kernel size
    :param stride: stride
    :param padding: padding
    :param bias: Bias
    :param node: 神经元类型
    :param kwargs:
    """

    def __init__(self,
                 in_channels: int,
                 out_channels: int,
                 kernel_size=(3, 3),
                 stride=(1, 1),
                 padding=(1, 1),
                 bias=False,
                 node=LIFNode,
                 step=6,
                 **kwargs):

        super().__init__()

        if node is None:
            raise TypeError

        self.groups = kwargs['groups'] if 'groups' in kwargs else 1
        self.conv = nn.Conv2d(in_channels=in_channels * self.groups,
                              out_channels=out_channels * self.groups,
                              kernel_size=kernel_size,
                              padding=padding,
                              stride=stride,
                              bias=bias)
        self.gn = nn.GroupNorm(16, out_channels * self.groups)
        self.node = partial(node, **kwargs)()
        self.step = step
        self.activation = nn.Identity()

    def forward(self, x):
        x = rearrange(x, '(t b) c w h -> t b c w h', t=self.step)
        outputs = []

        for t in range(self.step):
            outputs.append(self.gn(self.conv(x[t])))
        outputs = torch.stack(outputs)  # t b c w h
        outputs = rearrange(outputs, 't b c w h -> (t b) c w h')
        outputs = self.node(outputs)

        return outputs


class LayerWiseLinearModule(nn.Module):
    """
    线性模块
    :param in_features: 输入尺寸
    :param out_features: 输出尺寸
    :param bias: 是否有Bias, 默认 ``False``
    :param node: 神经元类型, 默认 ``LIFNode``
    :param args:
    :param kwargs:
    """

    def __init__(self,
                 in_features: int,
                 out_features: int,
                 bias=True,
                 node=LIFNode,
                 step=6,
                 spike=False,
                 *args,
                 **kwargs):
        super().__init__()
        if node is None:
            raise TypeError

        self.groups = kwargs['groups'] if 'groups' in kwargs else 1
        if self.groups == 1:
            self.fc = nn.Linear(in_features=in_features,
                                out_features=out_features, bias=bias)
        else:
            self.fc = nn.ModuleList()
            for i in range(self.groups):
                self.fc.append(nn.Linear(
                    in_features=in_features,
                    out_features=out_features,
                    bias=bias
                ))
        self.node = partial(node, **kwargs)()
        self.step = step
        self.spike = spike

    def forward(self, x):
        if self.groups == 1:  # (t b) c
            x = rearrange(x, '(t b) c -> t b c', t=self.step)
            outputs = []
            for t in range(self.step):
                outputs.append(self.fc(x[t]))
            outputs = torch.stack(outputs)  # t b c
            outputs = rearrange(outputs, 't b c -> (t b) c')

        else:  # b (c t)
            x = rearrange(x, 'b (c t) -> t b c', t=self.groups)
            outputs = []
            for i in range(self.groups):
                outputs.append(self.fc[i](x[i]))
            outputs = torch.stack(outputs)  # t b c
            outputs = rearrange(outputs, 't b c -> b (c t)')
        if self.spike:
            return self.node(outputs)
        else:
            return outputs


class LayWiseConvNet(BaseConvNet):
    def __init__(self,
                 step,
                 input_channels,
                 num_classes,
                 encode_type,
                 spike_output: bool,
                 out_channels: list,
                 node_list: list,
                 block_depth: list,
                 *args,
                 **kwargs):
        super().__init__(step,
                         input_channels,
                         num_classes,
                         encode_type,
                         spike_output,
                         out_channels,
                         block_depth,
                         node_list,
                         *args,
                         **kwargs)

    def _create_feature(self):
        feature_depth = len(self.node_list) - 1

        feature = [LayerWiseConvModule(
            self.input_channels * self.init_channel_mul, self.out_channels[0], node=self.node_list[0],
            groups=self.groups, step=self.step)]
        if self.block_depth[0] != 1:
            feature.extend(
                [LayerWiseConvModule(self.out_channels[0], self.out_channels[0], node=self.node_list[0],
                                     groups=self.groups, step=self.step)] * (
                        self.block_depth[0] - 1),
            )
        feature.append(TEP(channel=self.out_channels[0], step=self.step))
        feature.append(nn.AvgPool2d(kernel_size=4, stride=2))
        for i in range(1, feature_depth - 1):
            feature.append(LayerWiseConvModule(
                self.out_channels[i - 1], self.out_channels[i], node=self.node_list[i], groups=self.groups,
                step=self.step))
            if self.block_depth[i] != 1:
                feature.extend(
                    [LayerWiseConvModule(self.out_channels[i], self.out_channels[i], node=self.node_list[i],
                                         groups=self.groups,
                                         step=self.step)] * (
                            self.block_depth[i] - 1),
                )
            feature.append(TEP(channel=self.out_channels[i], step=self.step))
            feature.append(nn.AvgPool2d(kernel_size=4, stride=2))
        feature.append(LayerWiseConvModule(
            self.out_channels[-3], self.out_channels[-2], node=self.node_list[-2], groups=self.groups,
            step=self.step))
        if self.block_depth[feature_depth - 1] != 1:
            feature.extend(
                [LayerWiseConvModule(self.out_channels[-2], self.out_channels[-2], node=self.node_list[-2],
                                     groups=self.groups,
                                     step=self.step)] * (
                        self.block_depth[feature_depth - 1] - 1),
            )
        feature.append(nn.AdaptiveAvgPool2d(1))

        return nn.Sequential(*feature)

    def _create_fc(self):
        fc = nn.Sequential(
            # NDropout(.5),
            LayerWiseLinearModule(
                self.out_channels[-2], self.out_channels[-1], node=self.node_list[-1], groups=self.groups,
                step=self.step, spike=False)
        )
        return fc


@register_model
def cifar_convnet(step,
                encode_type,
                spike_output: bool,
                node_type,
                *args,
                **kwargs):
    # out_channels = [256, 256, 512, 1024]
    out_channels = [64, 128, 128, 256]
    block_depth = [2, 2, 2, 2]
    # print(kwargs)
    node_cls = partial(node_type, step=step, **kwargs)
    # print(node_cls)
    if spike_output:
        node_list = [node_cls] * (len(out_channels) + 1)
    else:
        node_list = [node_cls] * (len(out_channels))

    return LayWiseConvNet(step=step,
                          input_channels=3,
                          encode_type=encode_type,
                          node_list=node_list,
                          block_depth=block_depth,
                          out_channels=out_channels,
                          spike_output=spike_output,
                          **kwargs)


@register_model
def dvs_convnet(step,
                encode_type,
                spike_output: bool,
                node_type,
                num_classes,
                *args,
                **kwargs):
    out_channels = [64, 128, 256, 512, 1024]
    block_depth = [2, 1, 2, 1, 2]

    node_cls = partial(node_type, step=step, **kwargs)
    if spike_output:
        node_list = [node_cls] * (len(out_channels) + 1)
        # node_list[-2] = partial(DoubleSidePLIFNode, step=step, **kwargs)
    else:
        node_list = [node_cls] * (len(out_channels))
        # node_list[-1] = partial(DoubleSidePLIFNode, step=step, **kwargs)

    return LayWiseConvNet(step=step,
                          input_channels=2,
                          num_classes=num_classes,
                          encode_type=encode_type,
                          node_list=node_list,
                          block_depth=block_depth,
                          out_channels=out_channels,
                          spike_output=spike_output,
                          **kwargs)


@register_model
class SimpleSNN(BaseModule, abc.ABC):
    def __init__(self,
                 channel=1,
                 num_classes=10,
                 step=8,
                 node_type=LIFNode,
                 encode_type='direct',
                 *args,
                 **kwargs):
        super().__init__(step, encode_type, *args, **kwargs)
        self.num_classes = num_classes

        self.node = node_type
        init_channel = channel

        self.feature = nn.Sequential(
            LayerWiseConvModule(init_channel, 32, kernel_size=7, padding=0, node=self.node, step=step),
            TEP(step=step, channel=32),
            nn.AvgPool2d(kernel_size=2, stride=2),
            LayerWiseConvModule(32, 64, kernel_size=4, padding=0, node=self.node, step=step),
            TEP(step=step, channel=64),
            nn.AvgPool2d(kernel_size=2, stride=2),
        )
        self.fc = nn.Sequential(
            nn.Flatten(),
            LayerWiseLinearModule(64 * 4 * 4, self.num_classes, node=self.node, spike=False, step=step),
        )

    def forward(self, inputs):
        inputs = self.encoder(inputs)
        self.reset()

        if self.layer_by_layer:
            x = self.feature(inputs)
            x = self.fc(x)
            x = rearrange(x, '(t b) c -> t b c', t=self.step).mean(0)
            return x

        else:
            outputs = []
            for t in range(self.step):
                x = inputs[t]
                x = self.feature(x)
                x = self.fc(x)
                outputs.append(x)

            return sum(outputs) / len(outputs)

In [ ]:
# Precomputed characteristics of the dataset dataset
torch.cuda.manual_seed(3154)

batch_size = 512
MAX_PHYSICAL_BATCH_SIZE = 32
target_ep = 8
c = 6
epochs = 1

step = 10
delta = 1e-5
devices = 4
r = 1
device = torch.device('cuda') if torch.cuda.is_available() else 'cpu'
# device = 'cpu'
disable_noise = False
data_root = "./dataset"
kwargs = {"num_workers": 1, "pin_memory": True}
dataset = 'NMNIST'
# NMNIST, cifar10, dvs_cifar10, MNIST, FashionMNIST


def train(model, device, train_loader, optimizer, epoch, privacy_engine):
    criterion = nn.CrossEntropyLoss().to(device)
    losses = []
    model.train()
    correct = 0
    for _batch_idx, (data, target) in enumerate(train_loader):
        # print(target)
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        losses.append(loss.item())
        pred = output.argmax(
            dim=1, keepdim=True
        )  # get the index of the max log-probability
        correct += pred.eq(target.view_as(pred)).sum().item()

    if not disable_noise:
        epsilon = privacy_engine.get_epsilon(delta=delta)
        print(
            f"Train Epoch: {epoch} \t"
            f"Loss: {np.mean(losses):.6f} "
        )
        print("Accuracy: {}/{} ({:.2f}%)\n".format(
            correct,
            len(train_loader.dataset),
            100.0 * correct / len(train_loader.dataset), ))
        print(
              f"(ε = {epsilon:.2f}, δ = {delta})"
              )
    else:
        print(f"Train Epoch: {epoch} \t Loss: {np.mean(losses):.6f}")

    return 100.0 * correct / len(train_loader.dataset)


def test(model, device, test_loader):
    model.eval()
    criterion = nn.CrossEntropyLoss().to(device)
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += criterion(output, target).item()  # sum up batch loss
            pred = output.argmax(
                dim=1, keepdim=True
            )  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader)

    print(
        "\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n".format(
            test_loss,
            correct,
            len(test_loader.dataset),
            100.0 * correct / len(test_loader.dataset),
        )
    )
    return correct / len(test_loader.dataset)


def run():
    if dataset == 'dvs_cifar10':
        train_loader, test_loader, train_data, test_data = load_dvs10_data(batch_size=batch_size, step=step)
        # train_loader, test_loader, _, _ = get_dvsc10_data(batch_size=batch_size, step=step)
    elif dataset == 'NMNIST':
        train_loader, test_loader, train_data, test_data = load_nmnist_data(batch_size=batch_size, step=step)
    else:
        train_data, test_data, train_loader, test_loader = load_static_data(data_root, batch_size, dataset)

    result = []
    result_train = []
    for _ in range(r):
        if dataset == 'cifar10':
            model = cifar_convnet(
                step=step,
                encode_type='direct',
                node_type=LIFNode,
                num_classes=10,
                spike_output=False,
                layer_by_layer=True,
                act_fun=QGateGrad
            )
            model.to(device)
            optimizer = optim.AdamW(model.parameters(), lr=0.001)
            scheduler = optim.lr_scheduler.MultiStepLR(optimizer, milestones=[40], gamma=0.1, last_epoch=-1)

        elif dataset == 'dvs_cifar10':
            model = dvs_convnet(
                step=step,
                encode_type='direct',
                node_type=LIFNode,
                num_classes=10,
                spike_output=False,
                layer_by_layer=True,
                act_fun=QGateGrad
            )
            model.to(device)
            optimizer = optim.AdamW(model.parameters(), lr=0.001)
            scheduler = optim.lr_scheduler.MultiStepLR(optimizer, milestones=[15], gamma=1, last_epoch=-1)
        elif dataset == 'NMNIST':
            model = SimpleSNN(
                channel=2,
                step=step,
                node_type=LIFNode,
                act_fun=QGateGrad,
                layer_by_layer=True,
            )
            model.to(device)

            optimizer = optim.AdamW(model.parameters(), lr=0.005)
            scheduler = optim.lr_scheduler.MultiStepLR(optimizer, milestones=[10], gamma=0.1, last_epoch=-1)

        elif dataset == 'MNIST' or dataset == 'FashionMNIST':
            model = SimpleSNN(
                channel=1,
                step=step,
                node_type=LIFNode,
                act_fun=QGateGrad,
                layer_by_layer=True,
            )
            model.to(device)
            optimizer = optim.AdamW(model.parameters(), lr=0.005)
            scheduler = optim.lr_scheduler.MultiStepLR(optimizer, milestones=[10], gamma=0.1, last_epoch=-1)
        if not disable_noise:
            privacy_engine = PrivacyEngine()
            model, optimizer, data_loader = privacy_engine.make_private_with_epsilon(
                module=model,
                optimizer=optimizer,
                data_loader=train_loader,
                max_grad_norm=c,
                epochs=epochs,
                target_delta=delta,
                target_epsilon=target_ep
            )
            with BatchMemoryManager(
                    data_loader=data_loader,
                    max_physical_batch_size=MAX_PHYSICAL_BATCH_SIZE,
                    optimizer=optimizer
            ) as memory_safe_data_loader:
                # if 1:
                for epoch in range(1, epochs + 1):
                    result_train.append(train(model, device, memory_safe_data_loader, optimizer, epoch, privacy_engine))
                    result.append(test(model, device, test_loader))
                    scheduler.step()
        else:
            privacy_engine = PrivacyEngine()
            model, optimizer, data_loader = privacy_engine.make_private(
                module=model,
                optimizer=optimizer,
                data_loader=train_loader,
                max_grad_norm=c,
                noise_multiplier=0.0,
            )
            with BatchMemoryManager(
                    data_loader=data_loader,
                    max_physical_batch_size=MAX_PHYSICAL_BATCH_SIZE,
                    optimizer=optimizer
            ) as memory_safe_data_loader:
                for epoch in range(1, epochs + 1):
                    train(model, device, memory_safe_data_loader, optimizer, epoch, privacy_engine)
                    result.append(test(model, device, test_loader))
                    scheduler.step()
    result = np.array(result).reshape((r, -1))
    result_train = np.array(result_train).reshape((r, -1))
    best_acc = np.mean(np.max(result, axis=1))
    print(best_acc)
    # np.save(file=f'./{dataset}/MP_test.npy', arr=result)
    # np.save(file=f'./{dataset}/MP_train.npy', arr=result_train)

    return model


model = run()


In [ ]:
train, test, _, _ = load_nmnist_data( 1_000, 100)

  0%|          | 0/1011893601 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
x, y = next(iter(train))

In [ ]:
x.shape

In [ ]:
x_, y_ = next(iter(test))

In [ ]:
dic = {}

for i in range(8):
    rand_train = np.random.randint(0, 1_000, 1)[0]

    train_data = x[rand_train, :, :, :]
    train_target = y[rand_train]
    for j in range(2):
        name = 'group.'+str(i)
        rand_test = np.random.randint(0, 1_000, 9 if j == 0 else 10)
        data, target = 0, 0
        if j == 0:
            name += '.0.'
            data = torch.cat((x_[rand_test, :, :, :, :], x[rand_train, :, :, : , :].unsqueeze(dim = 0)), dim = 0)
            target = torch.cat((y_[rand_test], y[rand_train].unsqueeze(dim = 0)))
        else:
            name += '.1.'
            data = x_[rand_test, :, :, :, :]
            target = y_[rand_test]

        dic[name + 'data'] = data
        dic[name + 'target'] = target



In [ ]:
dic['group.0.0.data'].shape, dic['group.0.0.target']

In [ ]:
dic['group.1.0.data'].shape, dic['group.1.0.target']

In [ ]:

loss = nn.CrossEntropyLoss().to(device)
model.eval()

with torch.inference_mode():
    for i in range(8):
        name = 'group.'+ str(i)
        for j in range(2):
            X = dic[name + '.'+str(j)+'.data'].to(device)
            Y = dic[name + '.'+str(j)+'.target'].to(device)

            output = model(X)

            loss_val = loss(output, Y)
            print(loss_val.item())

        print('\n')


In [ ]:
device = 'cpu'

In [ ]:
criterion = nn.CrossEntropyLoss().to(device)

for i in range(8):
  for j in range(2):
    group = 'group_'+str(i)+'.'+str(j)

    if j == 0:

      images = data[group][0][1:]
      targets = data[group][1][1:]
    else:
      images = data[group][0]
      targets = data[group][1]

    group_data = Test_dataloader(images, targets)

    group_loader = DataLoader(test, batch_size=10, shuffle = True)

    model.eval()
    with torch.inference_mode():
      for X, y in group_loader:
        X, y = X.to(device), y.to(device)

        output = model(X)

        loss_val += criterion(output, y).item()  #
      print(group, loss_val.item())
      if group[-1] == 0:
        lst0.append(loss_val)
      else:
        lst1.append(loss_val)

  print()
  print()